In [1]:
from tqdm import tqdm
import pandas as pd
import os
import gzip
import zipfile
import shutil
from time import sleep
import pickle
import tarfile
import bz2
import sys

In [2]:
#t = tarfile.open("2017.tar")
#ids=0
#for filename in tqdm(t.getnames()):
#    if filename!='2017' and filename!='2017/_SUCCESS':
#        f=t.extractfile(filename)
#        source_file = bz2.BZ2File(f, "r")
#        for line in source_file:
#            break

In [3]:
with open('url_id.pickle', 'rb') as file:
    url_id = pickle.load(file)
with open('query_dict/querry_id.pickle', 'rb') as file:
    querry_id = pickle.load(file)
with open('query_dict/enlarged_querrys.pickle', 'rb') as file:
    enlarged_querrys = pickle.load(file)

In [4]:
def http_cut(ex_str):
    ex_str = ex_str.replace('http://',"")
    ex_str = ex_str.replace('https://',"")
    ex_str = ex_str.replace('www.',"")
    if len(ex_str)>0:
        if ex_str[-1]=='/':
            ex_str=ex_str[:-1]
    return ex_str

In [5]:
url_id_std = dict()
id_url_std = dict()

In [6]:
for key in tqdm(url_id):
    new_key = http_cut(key)
    url_id_std[new_key] = url_id[key]
    id_url_std[url_id[key]] = new_key

100%|██████████| 582167/582167 [00:01<00:00, 338849.64it/s]


In [7]:
with open('url_id_std.pickle', 'wb') as file:
    pickle.dump(url_id_std, file)
with open('id_url_std.pickle', 'wb') as file:
    pickle.dump(id_url_std, file)

In [8]:
url_id = None

In [9]:
min_timestamp = 1483218001000
raw_data = []
idx = -1
str_index = -1
t = tarfile.open("2017.tar")
for filename in tqdm(t.getnames()):
    #try:
    if filename!='2017' and filename!='2017/_SUCCESS':
        f=t.extractfile(filename)
        source_file = bz2.BZ2File(f, "r")
        idx+=1
        for line in source_file:
            flag = False
            cur_data = []
            a,b,c,d = line.decode('utf-8', errors = 'ignore').strip('\n').split('\t')
            tmp = a.split('@')
            querry = tmp[0]
            if querry in enlarged_querrys:
                cur_data.append(enlarged_querrys[querry])
                flag = True
            else:
                cur_data.append(-1)
            geo = int(tmp[1])
            cur_data.append(geo)
            shown_urls = b.split(',')
            shown_urls = [http_cut(word) for word in shown_urls]
            url_list = []
            loc_bad_dict = dict()
            loc_bad_idx=-1
            for elem in shown_urls:
                if elem in url_id_std:
                    url_list.append(url_id_std[elem])
                    flag = True
                else:
                    loc_bad_dict[elem] = loc_bad_idx
                    url_list.append(loc_bad_idx)
                    loc_bad_idx-=1
            cur_data.append(tuple(url_list))
            clicked_urls = c.split(',')
            clicked_urls = [http_cut(word) for word in clicked_urls]
            url_list = []
            for elem in clicked_urls:
                if elem in url_id_std:
                    url_list.append(url_id_std[elem])
                else:
                    url_list.append(loc_bad_dict[elem])
            cur_data.append(tuple(url_list))
            timestamps = [int(el) for el in d.split(',')]
            cur_data.append(tuple(timestamps))
            if flag:
                str_index+=1
                cur_data.append(str_index)
                raw_data.append(tuple(cur_data))
        if raw_data==[]:
            print(idx, filename, 'no relevant info')
        with open('raw_{}.pickle'.format(idx), 'wb') as file:
            pickle.dump(tuple(raw_data), file)
        zip_obj= zipfile.ZipFile("raw_{}.zip".format(idx),"w", compression=zipfile.ZIP_DEFLATED)
        zip_obj.write("raw_{}.pickle".format(idx))
        zip_obj.close()
        shutil.copy2("raw_{}.zip".format(idx), 'raw_click_data_upd/'+"raw_{}.zip".format(idx))
        try:
            os.remove("raw_{}.zip".format(idx))
            os.remove("raw_{}.pickle".format(idx))
        except PermissionError:
            sleep(3)
            os.remove("raw_{}.zip".format(idx))
            os.remove("raw_{}.pickle".format(idx))
        raw_data = []
    #except:
        #print(filename)
        #continue
t.close()

 34%|███▍      | 649/1902 [6:18:28<28:34:28, 82.10s/it] 

648 2017/part-m-01693.bz2 no relevant info


 38%|███▊      | 718/1902 [7:08:27<10:38:11, 32.34s/it]

717 2017/part-m-01695.bz2 no relevant info


 43%|████▎     | 818/1902 [8:04:11<6:18:20, 20.94s/it] 

...

 83%|████████▎ | 1578/1902 [13:52:35<1:26:56, 16.10s/it]

1576 2017/part-m-01696.bz2 no relevant info


 90%|████████▉ | 1704/1902 [14:32:00<1:04:30, 19.55s/it]

1702 2017/part-m-01694.bz2 no relevant info


100%|██████████| 1902/1902 [15:35:11<00:00, 29.50s/it]  
